In [1203]:
import pandas as pd
import glob
import numpy as np
import pathlib as pl
import os

### Check non-PKS datasets one by one

In [1204]:
PATH_TO_PROCESSED_DATA = pl.Path('../../datasets/processed/non-pks')


all_non_pks_files = glob.glob("../../datasets/raw/*.csv")
all_non_pks_files

['../../datasets/raw\\consumer_price_index_2000_2023.csv',
 '../../datasets/raw\\foreigners_in_germany_2000_2023.csv',
 '../../datasets/raw\\hospitals_general_2000_2023.csv',
 '../../datasets/raw\\income_and_expenditure_of_households_2000_2022.csv',
 '../../datasets/raw\\migration_total_2000_2023.csv',
 '../../datasets/raw\\migration_total_2008_2023_by_age.csv',
 '../../datasets/raw\\net_income_risk_of_poverty2008_2019_1_2.csv',
 '../../datasets/raw\\net_income_risk_of_poverty2020-2023_2_2.csv',
 '../../datasets/raw\\persons_in_employment_2008_2024.csv',
 '../../datasets/raw\\school_graduates_and_dropouts_2000_2023.csv',
 '../../datasets/raw\\suspects_1993-2022.csv',
 '../../datasets/raw\\total_criminal_offences_1993_2022.csv',
 '../../datasets/raw\\total_number_of_suspects_1993_2022.csv',
 '../../datasets/raw\\unemployment_2008_2024.csv']

##### Helper functions

In [1237]:
def purge_est_cols(df):
    for col in df.columns:
        if col.endswith("est"):
            df = df.drop([col],axis=1)

    return df


def mark_dupe_cols(df):
    d = {}
    new_cols = []
    for col in df.columns:
        if col in d.keys():
            new_cols.append(col + "_dupe")
        else:
            d[col] = True   
            new_cols.append(col)  
    df.columns = new_cols
    return df

def purge_dupe(df):
    for col in df.columns:
        if col.endswith("_dupe"):
            df = df.drop([col],axis=1)

    return df

def replace_unnamed_with_est(df):
    for (i, col) in enumerate(df.columns):
        if (col.startswith("Unnamed")) and i != 0:
            df.rename(columns={col: df.columns[i-1] + "_est"}, inplace=True)

    return df


def ffill_header(df, header_lvl):
    new_cols = [[0] ]* header_lvl
    for i in range(header_lvl):
        row = df.columns.get_level_values(i)
        new_row = [""] * len(row)
        good_val_met = False
        fill_word = ""
        for j in range(len(row)):
            name = row[j]

            if name.startswith("Unnamed"):
                if good_val_met:
                    new_row[j] = fill_word
                else:
                    new_row[j] = "Year" + str(j) + str(i)
            else:
                good_val_met = True
                fill_word = name
                new_row[j] = name
        new_cols[i] = new_row
    df.columns = new_cols
    return df

def set_type_to_float_from(df, col_to_start):
    for (i,col) in enumerate(df.columns):
        if i>=col_to_start:
            df[col] = df.loc[:,col].astype(float)

    return df
#first column is int (Year), rest is float
def normal_type_setup(df):
    df[df.columns[0]] = df.loc[:,df.columns[0]].astype(int)
    df = set_type_to_float_from(df,1)
    return df

def allint_type_setup(df):
    for col in df.columns:
        df[col] = df.loc[:,col].astype(int)
    return df
def set_type_to_int_from(df, col_to_start):
    for (i,col) in enumerate(df.columns):
        if i>=col_to_start:
            df[col] = df.loc[:,col].astype(int)

    return df

def prepend_col_name(df, name):
    for (i, col) in enumerate(df.columns):
        if i != 0:
            df.rename(columns={col: name + "_" +df.columns[i]}, inplace=True)


    return df

def new_column_from_cols_and_specif(col, specifier):
    col = ''.join(list(map(lambda x: str(x).strip("\n").strip("<NA>"), col)))
    clean_spec = str(specifier).strip('\n')
    new_col = f"{clean_spec}: {col}"
    new_col = new_col.replace("nan", "").replace("\\n", "")
    return new_col


def create_new_cols(specifier_column, other_columns):
    return [new_column_from_cols_and_specif(col, specifier)
                for specifier in specifier_column
                for col in other_columns
            ]

def create_transformed_df(df, specifier_column, other_columns, year):
    new_columns = create_new_cols(specifier_column, other_columns)

    # Flatten the values of all columns except the specifier column
    new_values = df.loc[:, other_columns].to_numpy().flatten()
    idx = np.array([np.issubdtype(type(x), np.floating) for x in new_values])
    filtered_values = new_values[idx]

    df_transformed = pd.DataFrame([new_values], columns=new_columns)
    df_transformed.insert(0, "Year", [year])
    df_transformed[df_transformed.columns[0]] = df_transformed.loc[:,df_transformed.columns[0]].astype(int)
    return df_transformed

def transform_one_df(df, specifier_key, interesting_values_basic_table, year):
    df = df[df[specifier_key].isin(interesting_values_basic_table)]

    specifier_column = df[specifier_key].to_numpy().flatten()
    other_columns = df.columns[df.columns != specifier_key]

    return create_transformed_df(df, specifier_column, other_columns, year)

def transform_all_years(df, specifier_column, other_columns):
    start_year = min(df[df.columns[0]])
    end_year = max(df[df.columns[0]])
    new_df = pd.DataFrame()
    for i in range(start_year, end_year+1):
        new_df = pd.concat([new_df, create_transformed_df(df.loc[df.Year==i,:], df.loc[df.Year==i,specifier_column], other_columns, i)],axis=0)

       
    return new_df

def mass_join(datasets):
    last_seen_df = None

    for tup in datasets:
        curr_df = tup[0]
        if curr_df is None:
            continue
        if last_seen_df is None:
            last_seen_df = curr_df
        else:
            last_seen_df = last_seen_df.merge(curr_df, on="Year", how="inner")
    return last_seen_df

def mass_join_till_year_x(datasets, year):
    last_seen_df = None

    for tup in datasets:
        curr_df = tup[0]
        if curr_df is None:
            continue
        if max(curr_df["Year"]) < year:
            continue
        if last_seen_df is None:
            last_seen_df = curr_df
        else:
            last_seen_df = last_seen_df.merge(curr_df, on="Year", how="inner")
    return last_seen_df

        


### Get all of the datasets loaded and prepared
Loads the datasets and does some basic cleaning. The prepared datasets are stored in the _datasets_ list, in the form _datasets[i] = (DataFrame, filepath to csv)_. 

The data is taken year by year, as the majority of the datasets do not have monthly info. Many of datasets of an indicator column for each numerical column, which has additional information about the value (e.g. whether it is an estimate or not). I assumed we would not need them so I stripped them for now. Other problems are discussed on a case by case basis.

#### Consumer Price Index

In [1206]:
datasets = []
proposed_changes = True
current_file = 0
df = pd.read_csv(all_non_pks_files[current_file], skiprows=6, sep=';', usecols=[0,1,3],  names=["Year", "Consumer price index", "CSI_Annual change (%)"])
datasets.append((df, all_non_pks_files[current_file]))
df.drop(df.tail(3).index, inplace=True)
df

,Year,Consumer price index,CSI_Annual change (%)
0,2000,75.5,1.3
1,2001,77.0,2.0
2,2002,78.1,1.4
3,2003,78.9,1.0
4,2004,80.2,1.6
5,2005,81.5,1.6
6,2006,82.8,1.6
7,2007,84.7,2.3
8,2008,86.9,2.6
9,2009,87.2,0.3


#### Foreigners in Germany
Has more entries per year documenting the regions of origin. In some of the years there are no records for some regions, denoted by a value of "-". Proposal: focus on the total number of foreigners instead, region analysis is going to be more difficult.


In [1207]:
#Foreigners in Germany 1
current_file = 1
df = pd.read_csv(all_non_pks_files[current_file],  sep=";", skiprows=8 ,names=["Year", "Region", "Male", "Male_est", "Female", "Female_est", "Total", "Total_est"])
df.loc[:, "Year"] = df.loc[:, "Year"].ffill().str.slice(0,4)
df.loc[:, "Year"]
df = purge_est_cols(df)
df = df[~(df["Region"] == "videlicet:")]
datasets.append((df, all_non_pks_files[current_file]))
df.drop(df.tail(4).index, inplace=True)
df

,Year,Region,Male,Female,Total
0,2000,Total,3959376,3337441,7296817
2,2000,Europe,3155602,2702107,5857709
3,2000,EU-27 (since 2020-02-01),-,-,-
4,2000,EU-27 third countries (since 2020-02-01),-,-,-
5,2000,EU-28 (until 2020-01-31),1456057,1215062,2671119
...,...,...,...,...,...
1003,2023,Territory of former Yugoslavia,792825,710015,1502840
1004,2023,Territory of former Serbia and Montenegro,324905,291070,615975
1005,2023,Territory of the former Soviet Union,780850,1155265,1936115
1006,2023,Territory of former Czechoslovakia,63300,69475,132775


#### Hospitals
Removed the first row which contained the units of measurment for each column.

In [1208]:
#Hospitals 2
current_file = 2
df = pd.read_csv(all_non_pks_files[current_file], skiprows=4,  sep=";")
df=replace_unnamed_with_est(df)
df=purge_est_cols(df)
df.rename(columns={df.columns[0]:"Year"}, inplace=True)
df = df.loc[1:,:]
df.drop(df.tail(3).index, inplace=True)
datasets.append((df, all_non_pks_files[current_file]))
df

,Year,Hospitals,Beds,Beds per 100 000 inhabitants,Patients,Patients per 100 000 inhabitants,Occupancy / billing days,Average length of stay,Average occupancy of hospital beds
1,2000,2242,559651,681,17262929,21004,167789.0,9.7,81.9
2,2001,2240,552680,671,17325083,21041,163536.0,9.4,81.1
3,2002,2221,547284,664,17432272,21135,159937.0,9.2,80.1
4,2003,2197,541901,657,17295910,20960,153518.0,8.9,77.6
5,2004,2166,531333,644,16801649,20365,146746.0,8.7,75.5
6,2005,2139,523824,635,16539398,20056,143244.0,8.7,74.9
7,2006,2104,510767,620,16832883,20437,142251.0,8.5,76.3
8,2007,2087,506954,616,17178573,20883,142893.0,8.3,77.2
9,2008,2083,503360,613,17519579,21334,142535.0,8.1,77.4
10,2009,2084,503341,615,17817180,21762,142414.0,8.0,77.5


#### Income and expenditure of households
This dataset is more difficult, what data is contained in each row is described by the Type and Subtype columns, I don't know how to combine them well so it stays like this for now. Also has more entries per year. I also removed the Unit column which described the units.

In [1209]:
#Income and expenditure - 3
current_file = 3
df = pd.read_csv(all_non_pks_files[current_file], skiprows=6,  sep=";")
df.rename(columns={df.columns[0]:"Year",df.columns[1]:"Type",df.columns[2]:"Subtype",df.columns[3]:"Unit"}, inplace=True)
df = replace_unnamed_with_est(df)
df = purge_est_cols(df)
df.loc[:, "Year"] = df.loc[:, "Year"].ffill().str.slice(0,4)
df.loc[:, "Type"] = df.loc[:, "Type"].ffill()
df.drop(df.tail(3).index, inplace=True)
df.drop(["Unit"], axis=1, inplace=True)
datasets.append((df, all_non_pks_files[current_file]))
df

,Year,Type,Subtype,Households with 1 person,Households with 2 persons,Households with 3 persons,Households with 4 persons,Households with 5 or more persons,Total
0,2000,Households covered,NaN,1602.0,2106.0,870.0,1039.0,-,5757.0
1,2000,Households extrapolated,NaN,12666.0,11935.0,4939.0,3865.0,-,34777.0
2,2000,Average amount per household and month,Household gross income,1965.0,3458.0,4631.0,4787.0,-,3303.0
3,2000,Average amount per household and month,Household net income,1538.0,2763.0,3555.0,3616.0,-,2538.0
4,2000,Average amount per household and month,Disposable income and receipts,1572.0,2931.0,3635.0,3691.0,-,2642.0
...,...,...,...,...,...,...,...,...,...
135,2022,Average amount per household and month,Household gross income,3164.0,5930.0,7438.0,8847.0,9051,5296.0
136,2022,Average amount per household and month,Household net income,2439.0,4621.0,5569.0,6569.0,6928,4056.0
137,2022,Average amount per household and month,Disposable income and receipts,2479.0,4702.0,5677.0,6664.0,7032,4125.0
138,2022,Average amount per household and month,Household final consumption expenditure,1833.0,3239.0,3760.0,4322.0,4632,2846.0


#### Migration 
The column names are generated by compounding a 3-level header, so that is why they look the way they do. Without this it was hard to parse what each means.

In [1210]:
#Migration total 4
current_file = 4
df = pd.read_csv(all_non_pks_files[current_file], header=[5,6,7],  sep=";")
df = df.loc[1:,:]
df = ffill_header(df, 3)
df.columns = [f'{i}_{j}_{k}' for i, j,k in df.columns]
df.rename(columns={(df.columns[0]):"Year"}, inplace=True)
df.drop(df.tail(3).index, inplace=True)
df = mark_dupe_cols(df)
df = purge_dupe(df)
datasets.append((df, all_non_pks_files[current_file]))
df

,Year,Germans_Male_Arrivals,Germans_Male_Departures,Germans_Male_Migration balance,Germans_Female_Arrivals,Germans_Female_Departures,Germans_Female_Migration balance,Germans_Total_Arrivals,Germans_Total_Departures,Germans_Total_Migration balance,...,Foreigners_Total_Migration balance,Total_Male_Arrivals,Total_Male_Departures,Total_Male_Migration balance,Total_Female_Arrivals,Total_Female_Departures,Total_Female_Migration balance,Total_Total_Arrivals,Total_Total_Departures,Total_Total_Migration balance
1,2000,603223,561172,42051,590714,552104,38610,1193937,1113276,80661,...,86454,1065059,1004023,61036,912732,806653,106079,1977791,1810676,167115
2,2001,619022,574607,44415,610109,567526,42583,1229131,1142133,86998,...,188760,1106217,981032,125185,953821,803248,150573,2060038,1784280,275758
3,2002,597681,562979,34702,591712,559844,31868,1189393,1122823,66570,...,152775,1064669,984134,80535,931369,792559,138810,1996038,1776693,219345
4,2003,573846,554434,19412,565140,544601,20539,1138986,1099035,39951,...,102697,1003715,956268,47447,880118,784917,95201,1883833,1741185,142648
5,2004,579988,564641,15347,555268,543289,11979,1135256,1107930,27326,...,55218,1010173,990934,19239,864503,801198,63305,1874676,1792132,82544
6,2005,539175,548618,-9443,530520,537840,-7320,1069695,1086458,-16763,...,95717,951792,930436,21356,826074,768476,57598,1777866,1698912,78954
7,2006,519466,546920,-27454,512179,536627,-24448,1031645,1083547,-51902,...,74694,923149,923638,-489,791842,768561,23281,1714991,1692199,22792
8,2007,530607,559278,-28671,526001,552420,-26419,1056608,1111698,-55090,...,99003,944393,932859,11534,813566,781187,32379,1757959,1714046,43913
9,2008,541586,575834,-34248,540201,572368,-32167,1081787,1148202,-66415,...,10691,957989,1001563,-43574,828041,840191,-12150,1786030,1841754,-55724
10,2009,534139,552319,-18180,532441,554516,-22075,1066580,1106835,-40255,...,27542,966902,985150,-18248,835398,829863,5535,1802300,1815013,-12713


#### Migration by age
Almost the exact same thing as above, but column two is now age groups for the given year.

In [1211]:
#Migration by age - 5
current_file = 5
df = pd.read_csv(all_non_pks_files[current_file], header=[5,6,7],  sep=";")
df = df.loc[1:,:]
df = ffill_header(df, 3)
df.columns = [f'{i}_{j}_{k}' for i, j,k in df.columns]
df.rename(columns={(df.columns[0]):"Year", df.columns[1]:"Age group"}, inplace=True)
df.loc[:, "Year"] = df.loc[:, "Year"].ffill().str.slice(0,4)
df.drop(df.tail(3).index, inplace=True)
df = mark_dupe_cols(df)
df = purge_dupe(df)
datasets.append((df, all_non_pks_files[current_file]))
df

,Year,Age group,Germans_Male_Arrivals,Germans_Male_Departures,Germans_Male_Migration balance,Germans_Female_Arrivals,Germans_Female_Departures,Germans_Female_Migration balance,Germans_Total_Arrivals,Germans_Total_Departures,...,Foreigners_Total_Migration balance,Total_Male_Arrivals,Total_Male_Departures,Total_Male_Migration balance,Total_Female_Arrivals,Total_Female_Departures,Total_Female_Migration balance,Total_Total_Arrivals,Total_Total_Departures,Total_Total_Migration balance
1,2008,under 1 year,4338,3465,873,4044,3312,732,8382,6777,...,1494,5678,4031,1647,5326,3874,1452,11004,7905,3099
2,2008,1 year,8666,8593,73,8016,7901,115,16682,16494,...,1966,11075,10001,1074,10366,9286,1080,21441,19287,2154
3,2008,2 years,7115,7434,-319,6606,6955,-349,13721,14389,...,1328,9281,8936,345,8616,8301,315,17897,17237,660
4,2008,3 years,6367,6927,-560,6213,6690,-477,12580,13617,...,957,8365,8444,-79,8098,8099,-1,16463,16543,-80
5,2008,4 years,5403,6052,-649,5089,5650,-561,10492,11702,...,663,7263,7598,-335,6879,7091,-212,14142,14689,-547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1548,2023,92 years,219,245,-26,576,620,-44,795,865,...,-29,233,278,-45,615,669,-54,848,947,-99
1549,2023,93 years,177,185,-8,489,539,-50,666,724,...,-15,196,214,-18,522,577,-55,718,791,-73
1550,2023,94 years,140,152,-12,348,394,-46,488,546,...,-9,150,173,-23,371,415,-44,521,588,-67
1551,2023,95 years and over,245,270,-25,842,970,-128,1087,1240,...,-109,257,337,-80,885,1067,-182,1142,1404,-262


#### Net income, risk of poverty 2008-2019
Has more entries per year per each age group.

In [1212]:
#Net income and risk of poverty years 2009-2019  6 
current_file = 6
df = pd.read_csv(all_non_pks_files[current_file], skiprows=6, sep=";")
df = df.loc[1:,:]
df.rename(columns={(df.columns[0]):"Year", df.columns[1]:"Sex", df.columns[2]:"Age groups"}, inplace=True)
df.loc[:, "Year"] = df.loc[:, "Year"].ffill().str.slice(0,4)
df.loc[:, "Sex"] = df.loc[:, "Sex"].ffill()
df = replace_unnamed_with_est(df)
df = purge_est_cols(df)
df.drop(df.tail(3).index, inplace=True)

datasets.append((df, all_non_pks_files[current_file]))
df


,Year,Sex,Age groups,Persons covered,Persons extrapolated,Equivalised net income (median),At-risk-of-poverty rate
1,2008,Male,16 to under 18 years,348,968.0,17723,19.0
2,2008,Male,18 to under 25 years,960,3510.0,18731,18.0
3,2008,Male,25 to under 50 years,4321,13873.0,20103,12.8
4,2008,Male,50 to under 65 years,2996,7518.0,20565,16.0
5,2008,Male,65 years and over,3072,7578.0,17147,12.0
...,...,...,...,...,...,...,...
212,2019,Total,18 to under 25 years,1285,6069.0,22099,18.8
213,2019,Total,25 to under 50 years,5975,26119.0,25188,12.3
214,2019,Total,50 to under 65 years,6772,19051.0,25537,16.1
215,2019,Total,65 years and over,6718,16640.0,20425,18.0


#### Net income, risk of poverty 2020-2023
This is apparently the continuation of the previous dataset, but in this they started measuring completely different metrics, so this one is by household types instead of age groups and gender.

In [1213]:
#Net income and risk of poverty years 2020-2023 7
current_file = 7
df = pd.read_csv(all_non_pks_files[current_file], skiprows=6, sep=";")
df = df.loc[1:,:]
df.rename(columns={(df.columns[0]):"Year", df.columns[1]:"Household type"}, inplace=True)
df.loc[:, "Year"] = df.loc[:, "Year"].ffill().str.slice(0,4)
df = replace_unnamed_with_est(df)
df = purge_est_cols(df)
df.drop(df.tail(3).index, inplace=True)

datasets.append((df, all_non_pks_files[current_file]))
df

,Year,Household type,Total,Persons at risk of poverty or exclusion,Persons at risk of poverty,Severely materially and socially deprived persons,Persons in househ. w. very low employm. intensity
1,2020,Households without children,48183.0,9994.0,8383.0,1865,2506.0
2,2020,Persons living alone,16468.0,5535.0,4728.0,1212,1608.0
3,2020,Two adults without children,25660.0,3694.0,3081.0,516,728.0
4,2020,Three or more adults without children,6054.0,765.0,575.0,137,170.0
5,2020,Households with children,33972.0,6741.0,4820.0,1720,2611.0
6,2020,Lone parents,3686.0,1729.0,1142.0,485,959.0
7,2020,Two adults with children,26350.0,4393.0,3234.0,1077,1441.0
8,2020,Three or more adults with children,3936.0,619.0,444.0,159,211.0
9,2020,Total,82175.0,16735.0,13203.0,3585,5118.0
10,2021,Households without children,47793.0,9907.0,7911.0,1903,2857.0


#### Persons in employment
Has entries by each quarter, possibly a target for shrinking into just years. Also "Type of values" has only 2 values - "Unadjusted values" and "X13 JDemetra+  seasonally adjusted", I do not really understand it, but maybe we could only use of them?

In [1214]:
#Persons in employment - 8
current_file = 8
df = pd.read_csv(all_non_pks_files[current_file], header=[6,7], sep=";")
df = df.loc[1:,:]

df = ffill_header(df, 2)
df.columns = [f'{i}_{j}' for i, j in df.columns]
df.rename(columns={df.columns[0]:"Type of values", df.columns[1]:"Year", df.columns[2]:"Quarter"}, inplace=True)
df = df.astype({"Year": "string"})
df.loc[:, "Year"] = df.loc[:, "Year"].ffill().str.slice(0,4)
df = mark_dupe_cols(df)
df = purge_dupe(df)
df.loc[:, "Type of values"] = df.loc[:, "Type of values"].ffill()
df.drop(df.tail(4).index, inplace=True)
datasets.append((df, all_non_pks_files[current_file]))
df

,Type of values,Year,Quarter,Place of work in Germany_Employees,Place of work in Germany_Rate of change on previous quarter,Place of work in Germany_Rate of change on previous year's quarter,Place of residence in Germany_Employees,Place of residence in Germany_Rate of change on previous quarter,Place of residence in Germany_Rate of change on previous year's quarter
1,Unadjusted values,2008,Quarter 1,35915,-1.1,1.9,35904,-0.9,1.9
2,Unadjusted values,2008,Quarter 2,36246,0.9,1.6,36160,0.7,1.7
3,Unadjusted values,2008,Quarter 3,36561,0.9,1.5,36432,0.8,1.6
4,Unadjusted values,2008,Quarter 4,36765,0.6,1.3,36714,0.8,1.4
5,Unadjusted values,2009,Quarter 1,36170,-1.6,0.7,36167,-1.5,0.7
...,...,...,...,...,...,...,...,...,...
131,X13 JDemetra+ seasonally adjusted,2023,Quarter 3,42188,0.1,x,41981,0.1,x
132,X13 JDemetra+ seasonally adjusted,2023,Quarter 4,42237,0.1,x,42029,0.1,x
133,X13 JDemetra+ seasonally adjusted,2024,Quarter 1,42289,0.1,x,42086,0.1,x
134,X13 JDemetra+ seasonally adjusted,2024,Quarter 2,42322,0.1,x,42121,0.1,x


#### School graduates and dropouts
3 entries per year - for Men, Women and the total amount.

In [1215]:
#School graduates and dropouts
print(current_file)
current_file = 9
df = pd.read_csv(all_non_pks_files[current_file], skiprows=5,  sep=";")
df = df.loc[1:,:]
df.rename(columns={(df.columns[1]):"Year", df.columns[0]:"Sex"}, inplace=True)
df.loc[:, "Year"] = df.loc[:, "Year"].ffill().str.slice(0,4)
df.loc[:, "Sex"] = df.loc[:, "Sex"].ffill()
df = replace_unnamed_with_est(df)
df = purge_est_cols(df)
df.drop(df.tail(3).index, inplace=True)

datasets.append((df, all_non_pks_files[current_file]))
df


8


,Sex,Year,Pupils,School beginners,School leavers (graduates and dropouts)
1,Male,2000,5062891,419055,473494
2,Male,2001,5016905,401771,476530
3,Male,2002,4969732,413783,483269
4,Male,2003,4941969,433303,500920
5,Male,2004,4889799,422289,488378
...,...,...,...,...,...
68,Total,2019,8326884,733443,749946
69,Total,2020,8380767,749820,768191
70,Total,2021,8436221,770682,769411
71,Total,2022,8693344,813540,782423


#### Suspects
Multi level header, otherwise nothing special.

In [1216]:
current_file = 10
df = pd.read_csv(all_non_pks_files[current_file], header=[5,6],  sep=";")
df = ffill_header(df, 2)
df.columns = [f'{i}_{j}' for i, j in df.columns]
df.rename(columns={(df.columns[0]):"Year"}, inplace=True)
df = mark_dupe_cols(df)
df = purge_dupe(df)
df.drop(df.tail(4).index, inplace=True)
df = allint_type_setup(df)
datasets.append((df, all_non_pks_files[current_file]))

df


,Year,Germans_Male,Germans_Female,Foreigners_Male,Foreigners_Female
0,1993,992608,302768,552382,115741
1,1994,1028234,317113,491962,100342
2,1995,1083993,336442,479194,101857
3,1996,1127829,352794,494045,107615
4,1997,1157673,365165,494694,111768
5,1998,1185583,380122,487841,113575
6,1999,1170218,368350,459938,114008
7,2000,1203643,374272,450260,112363
8,2001,1215912,378039,433074,110541
9,2002,1252792,394033,431672,113107


#### Total criminal offences

In [1217]:
current_file = 11
df = pd.read_csv(all_non_pks_files[current_file], header=[4],  sep=";")
df=df.loc[1:,:]
#df.columns = [f'{i}_{j}' for i, j in df.columns]
df.rename(columns={(df.columns[0]):"Year"}, inplace=True)
df = replace_unnamed_with_est(df)
df = purge_est_cols(df)
df.drop(df.tail(4).index, inplace=True)
df = allint_type_setup(df)
datasets.append((df, all_non_pks_files[current_file]))

df


,Year,Registered criminal offences,Cleared-up criminal offences
1,1993,6750613,2957135
2,1994,6537748,2899733
3,1995,6668717,3068379
4,1996,6647598,3255042
5,1997,6586165,3335016
6,1998,6456996,3376524
7,1999,6302316,3329124
8,2000,6264723,3335356
9,2001,6363865,3379618
10,2002,6507394,3425416


#### Total number of suspects 
This is the exact same csv as suspects_1993-2022.csv, skipping..

In [1218]:
current_file = 12

#### Unemployment
Has entries per month, should probably reduce it to years.

In [1219]:
current_file = 13
df = pd.read_csv(all_non_pks_files[current_file], header=[6,7], skipfooter=7,  sep=";")
df = ffill_header(df, 2)
df.columns = [f'{i}_{j}' for i, j in df.columns]
df.rename(columns={(df.columns[0]):"Year", df.columns[1]:"Month"}, inplace=True)
df=df.loc[1:,:]
df = mark_dupe_cols(df)
df = purge_dupe(df)
df.loc[:, "Year"] = df.loc[:, "Year"].ffill()
df.Year = df.Year.astype(int) 
#df.drop(df.tail(4).index, inplace=True)
for (i,col) in enumerate(df.columns):
    if (i > 1) :
        df.loc[:,col] = df.loc[:,col].astype(float)


if proposed_changes:
    df.drop(["Month"], inplace=True, axis=1)
    df.groupby("Year").mean()
datasets.append((df, all_non_pks_files[current_file]))
df
df.groupby("Year").mean()


C:\Users\danza\AppData\Local\Temp\ipykernel_1308\1432359443.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(all_non_pks_files[current_file], header=[6,7], skipfooter=7,  sep=";")


,Unadjusted values_Economically active population,Unadjusted values_Persons in employment,Unadjusted values_Unemployed persons,Unadjusted values_Unemployment rate,X13 JDemetra+ trend_Economically active population,X13 JDemetra+ trend_Persons in employment,X13 JDemetra+ trend_Unemployed persons,X13 JDemetra+ trend_Unemployment rate,BV4.1 trend_Economically active population,BV4.1 trend_Persons in employment,BV4.1 trend_Unemployed persons,BV4.1 trend_Unemployment rate
Year,,,,,,,,,,,,
2008,40.655833,37.6375,3.018333,7.425,40.61,37.6125,2.9975,7.4,40.599167,37.571667,3.0275,7.45
2009,39.894167,37.016667,2.8775,7.225,39.8975,37.001667,2.895833,7.241667,39.98,37.091667,2.888333,7.233333
2010,39.84,37.223333,2.616667,6.575,39.869167,37.255,2.614167,6.55,39.878333,37.268333,2.61,6.533333
2011,40.401667,38.174167,2.2275,5.516667,40.378333,38.146667,2.231667,5.525,40.333333,38.098333,2.235,5.55
2012,40.565833,38.500833,2.065,5.083333,40.5725,38.515833,2.056667,5.058333,40.593333,38.526667,2.066667,5.1
2013,40.9075,38.883333,2.024167,4.95,40.903333,38.8825,2.020833,4.941667,40.926667,38.905,2.021667,4.933333
2014,41.154167,39.213333,1.940833,4.708333,41.178333,39.2325,1.945833,4.716667,41.1475,39.215833,1.931667,4.708333
2015,41.331667,39.524167,1.8075,4.366667,41.345,39.5425,1.8025,4.358333,41.371667,39.575,1.796667,4.341667
2016,42.206667,40.559167,1.6475,3.908333,42.1725,40.515,1.6575,3.933333,42.150833,40.493333,1.6575,3.925


In [1220]:
print(len(datasets))
datasets[0]

13


(    Year  Consumer price index  CSI_Annual change (%)
 0   2000                  75.5                    1.3
 1   2001                  77.0                    2.0
 2   2002                  78.1                    1.4
 3   2003                  78.9                    1.0
 4   2004                  80.2                    1.6
 5   2005                  81.5                    1.6
 6   2006                  82.8                    1.6
 7   2007                  84.7                    2.3
 8   2008                  86.9                    2.6
 9   2009                  87.2                    0.3
 10  2010                  88.1                    1.0
 11  2011                  90.0                    2.2
 12  2012                  91.7                    1.9
 13  2013                  93.1                    1.5
 14  2014                  94.0                    1.0
 15  2015                  94.5                    0.5
 16  2016                  95.0                    0.5
 17  2017 

### Edit datasets into unified formats
Make sure everydataset is built so that the year column is a key so that they can be joined later.

#### Consumer Price Index
Just retyped year to int, no other changes.

In [1221]:
curr_dataset = 0
df = datasets[curr_dataset][0]
df.Year = df.loc[:,"Year"].astype(int)
datasets[curr_dataset] = (df, datasets[curr_dataset][1])
df


,Year,Consumer price index,CSI_Annual change (%)
0,2000,75.5,1.3
1,2001,77.0,2.0
2,2002,78.1,1.4
3,2003,78.9,1.0
4,2004,80.2,1.6
5,2005,81.5,1.6
6,2006,82.8,1.6
7,2007,84.7,2.3
8,2008,86.9,2.6
9,2009,87.2,0.3


#### Foreigners
Removed regions "EU-27 (since 2020-02-01)", "EU-27 third countries (since 2020-02-01)" and "United Kingdom incl. British overseas territories" since they seemed to have NaN's in all years. Also cutout years before 2004, since they had even more NaN's.

In [1222]:
curr_dataset = 1
df = datasets[curr_dataset][0]
df.Year = df.loc[:,"Year"].astype(int)

df.replace("-", np.nan, inplace=True)
#df.loc[:,"Female"] / df.loc[:,"Male"] 
mask1 = df["Region"] == "EU-27 (since 2020-02-01)"
mask2 = df["Region"] == "EU-27 third countries (since 2020-02-01)"
mask3 = df["Region"] == "United Kingdom incl. British overseas territories"
mask4 = df["Year"] <= 2003
df = df[~mask1]
df = df[~mask2]
df = df[~mask3]
df = df[~mask4]
df = set_type_to_int_from(df, 2)
df
#a = create_transformed_df(df.loc[df.Year == 2009,:], df.loc[df.Year == 2009,"Region"], ["Male", "Female", "Total"], 2009)
#b = create_transformed_df(df, ["Region"], ["Male", "Female", "Total"], 2008)
df = transform_all_years(df,"Region", ["Male", "Female", "Total"])
datasets[curr_dataset] = (df, datasets[curr_dataset][1])
prepend_col_name(df, "Foreigners")

df.dtypes



C:\Users\danza\AppData\Local\Temp\ipykernel_1308\3316352929.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~mask2]
C:\Users\danza\AppData\Local\Temp\ipykernel_1308\3316352929.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~mask3]
C:\Users\danza\AppData\Local\Temp\ipykernel_1308\3316352929.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~mask4]


Year                                                       int32
Foreigners_Total: Male                                     int32
Foreigners_Total: Female                                   int32
Foreigners_Total: Total                                    int32
Foreigners_Europe: Male                                    int32
                                                           ...  
Foreigners_Territory of the former Soviet Union: Female    int32
Foreigners_Territory of the former Soviet Union: Total     int32
Foreigners_Territory of former Czechoslovakia: Male        int32
Foreigners_Territory of former Czechoslovakia: Female      int32
Foreigners_Territory of former Czechoslovakia: Total       int32
Length: 115, dtype: object

#### Hospitals

In [1223]:
curr_dataset = 2
df = datasets[curr_dataset][0]
df["Year"] = df["Year"].astype(int)
datasets[curr_dataset] = (df, datasets[curr_dataset][1])
df.dtypes

Year                                    int32
Hospitals                              object
Beds                                   object
Beds per 100 000 inhabitants           object
Patients                               object
Patients per 100 000 inhabitants       object
Occupancy / billing days              float64
Average length of stay                 object
Average occupancy of hospital beds     object
dtype: object

#### Income and expenditure of households
The CSV is missing multiple years (2008, 2013, 2018, 2023) which is kinda terrible, skipping for now.

In [1224]:
curr_dataset = 3
df = datasets[curr_dataset][0]
datasets[curr_dataset] = (None, datasets[curr_dataset][1])

"""
df["Type"] = df["Type"] + "_" + df["Subtype"].replace(np.nan, "")
df["Type"] = df["Type"].str.replace("ed_", "ed")
df.drop("Subtype", axis=1, inplace=True)
df["Year"] = df["Year"].astype(int)
mask = df["Year"] <= 2007
df = df[~mask]
df["Households with 5 or more persons"] = df["Households with 5 or more persons"].astype(float)
df

transform_all_years(df, "Type", [df.columns[2],df.columns[3],df.columns[4],df.columns[5],df.columns[6],df.columns[7]])"""


'\ndf["Type"] = df["Type"] + "_" + df["Subtype"].replace(np.nan, "")\ndf["Type"] = df["Type"].str.replace("ed_", "ed")\ndf.drop("Subtype", axis=1, inplace=True)\ndf["Year"] = df["Year"].astype(int)\nmask = df["Year"] <= 2007\ndf = df[~mask]\ndf["Households with 5 or more persons"] = df["Households with 5 or more persons"].astype(float)\ndf\n\ntransform_all_years(df, "Type", [df.columns[2],df.columns[3],df.columns[4],df.columns[5],df.columns[6],df.columns[7]])'

#### Migration total

In [1225]:
curr_dataset = 4
df = datasets[curr_dataset][0]
df = allint_type_setup(df)
datasets[curr_dataset] = (df, datasets[curr_dataset][1])
df

,Year,Germans_Male_Arrivals,Germans_Male_Departures,Germans_Male_Migration balance,Germans_Female_Arrivals,Germans_Female_Departures,Germans_Female_Migration balance,Germans_Total_Arrivals,Germans_Total_Departures,Germans_Total_Migration balance,...,Foreigners_Total_Migration balance,Total_Male_Arrivals,Total_Male_Departures,Total_Male_Migration balance,Total_Female_Arrivals,Total_Female_Departures,Total_Female_Migration balance,Total_Total_Arrivals,Total_Total_Departures,Total_Total_Migration balance
1,2000,603223,561172,42051,590714,552104,38610,1193937,1113276,80661,...,86454,1065059,1004023,61036,912732,806653,106079,1977791,1810676,167115
2,2001,619022,574607,44415,610109,567526,42583,1229131,1142133,86998,...,188760,1106217,981032,125185,953821,803248,150573,2060038,1784280,275758
3,2002,597681,562979,34702,591712,559844,31868,1189393,1122823,66570,...,152775,1064669,984134,80535,931369,792559,138810,1996038,1776693,219345
4,2003,573846,554434,19412,565140,544601,20539,1138986,1099035,39951,...,102697,1003715,956268,47447,880118,784917,95201,1883833,1741185,142648
5,2004,579988,564641,15347,555268,543289,11979,1135256,1107930,27326,...,55218,1010173,990934,19239,864503,801198,63305,1874676,1792132,82544
6,2005,539175,548618,-9443,530520,537840,-7320,1069695,1086458,-16763,...,95717,951792,930436,21356,826074,768476,57598,1777866,1698912,78954
7,2006,519466,546920,-27454,512179,536627,-24448,1031645,1083547,-51902,...,74694,923149,923638,-489,791842,768561,23281,1714991,1692199,22792
8,2007,530607,559278,-28671,526001,552420,-26419,1056608,1111698,-55090,...,99003,944393,932859,11534,813566,781187,32379,1757959,1714046,43913
9,2008,541586,575834,-34248,540201,572368,-32167,1081787,1148202,-66415,...,10691,957989,1001563,-43574,828041,840191,-12150,1786030,1841754,-55724
10,2009,534139,552319,-18180,532441,554516,-22075,1066580,1106835,-40255,...,27542,966902,985150,-18248,835398,829863,5535,1802300,1815013,-12713


#### Migration total by age


In [1226]:
curr_dataset = 5
df = datasets[curr_dataset][0]
#Skip for now
datasets[curr_dataset] = (None, datasets[curr_dataset][1])
df

,Year,Age group,Germans_Male_Arrivals,Germans_Male_Departures,Germans_Male_Migration balance,Germans_Female_Arrivals,Germans_Female_Departures,Germans_Female_Migration balance,Germans_Total_Arrivals,Germans_Total_Departures,...,Foreigners_Total_Migration balance,Total_Male_Arrivals,Total_Male_Departures,Total_Male_Migration balance,Total_Female_Arrivals,Total_Female_Departures,Total_Female_Migration balance,Total_Total_Arrivals,Total_Total_Departures,Total_Total_Migration balance
1,2008,under 1 year,4338,3465,873,4044,3312,732,8382,6777,...,1494,5678,4031,1647,5326,3874,1452,11004,7905,3099
2,2008,1 year,8666,8593,73,8016,7901,115,16682,16494,...,1966,11075,10001,1074,10366,9286,1080,21441,19287,2154
3,2008,2 years,7115,7434,-319,6606,6955,-349,13721,14389,...,1328,9281,8936,345,8616,8301,315,17897,17237,660
4,2008,3 years,6367,6927,-560,6213,6690,-477,12580,13617,...,957,8365,8444,-79,8098,8099,-1,16463,16543,-80
5,2008,4 years,5403,6052,-649,5089,5650,-561,10492,11702,...,663,7263,7598,-335,6879,7091,-212,14142,14689,-547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1548,2023,92 years,219,245,-26,576,620,-44,795,865,...,-29,233,278,-45,615,669,-54,848,947,-99
1549,2023,93 years,177,185,-8,489,539,-50,666,724,...,-15,196,214,-18,522,577,-55,718,791,-73
1550,2023,94 years,140,152,-12,348,394,-46,488,546,...,-9,150,173,-23,371,415,-44,521,588,-67
1551,2023,95 years and over,245,270,-25,842,970,-128,1087,1240,...,-109,257,337,-80,885,1067,-182,1142,1404,-262


#### Net income risk of poverty

In [1227]:
curr_dataset =6 
df = datasets[curr_dataset][0]
datasets[curr_dataset] = (None, datasets[curr_dataset][1])
df

,Year,Sex,Age groups,Persons covered,Persons extrapolated,Equivalised net income (median),At-risk-of-poverty rate
1,2008,Male,16 to under 18 years,348,968.0,17723,19.0
2,2008,Male,18 to under 25 years,960,3510.0,18731,18.0
3,2008,Male,25 to under 50 years,4321,13873.0,20103,12.8
4,2008,Male,50 to under 65 years,2996,7518.0,20565,16.0
5,2008,Male,65 years and over,3072,7578.0,17147,12.0
...,...,...,...,...,...,...,...
212,2019,Total,18 to under 25 years,1285,6069.0,22099,18.8
213,2019,Total,25 to under 50 years,5975,26119.0,25188,12.3
214,2019,Total,50 to under 65 years,6772,19051.0,25537,16.1
215,2019,Total,65 years and over,6718,16640.0,20425,18.0


#### Net income risk of povery 2020 -

In [1228]:
curr_dataset = 7
df = datasets[curr_dataset][0]
datasets[curr_dataset] = (None, datasets[curr_dataset][1])
df

,Year,Household type,Total,Persons at risk of poverty or exclusion,Persons at risk of poverty,Severely materially and socially deprived persons,Persons in househ. w. very low employm. intensity
1,2020,Households without children,48183.0,9994.0,8383.0,1865,2506.0
2,2020,Persons living alone,16468.0,5535.0,4728.0,1212,1608.0
3,2020,Two adults without children,25660.0,3694.0,3081.0,516,728.0
4,2020,Three or more adults without children,6054.0,765.0,575.0,137,170.0
5,2020,Households with children,33972.0,6741.0,4820.0,1720,2611.0
6,2020,Lone parents,3686.0,1729.0,1142.0,485,959.0
7,2020,Two adults with children,26350.0,4393.0,3234.0,1077,1441.0
8,2020,Three or more adults with children,3936.0,619.0,444.0,159,211.0
9,2020,Total,82175.0,16735.0,13203.0,3585,5118.0
10,2021,Households without children,47793.0,9907.0,7911.0,1903,2857.0


#### Employment
Kept the X13 JDemetra+ seasonally adjusted data, dropped unadjusted values. Threw out rate of change columns.

In [1229]:
curr_dataset = 8
df = datasets[curr_dataset][0]
mask = df["Type of values"] == "Unadjusted values"
df = df[~mask]
df = df.drop([df.columns[4],df.columns[5],df.columns[7],df.columns[8], df.columns[0], df.columns[2]], axis=1)
df = allint_type_setup(df)
df = df.groupby("Year", as_index=False).mean().astype(int)
datasets[curr_dataset] = (df, datasets[curr_dataset][1])
df.dtypes


Year                                       int32
Place of work in Germany_Employees         int32
Place of residence in Germany_Employees    int32
dtype: object

#### School graduates and dropouts

In [1230]:
curr_dataset = 9
df = datasets[curr_dataset][0]
mask = df.Sex == "Total"
df = df[mask]
df.drop(["Sex"],axis=1, inplace=True)
df.drop(["School leavers (graduates and dropouts)"],axis=1, inplace=True)
df = allint_type_setup(df)
datasets[curr_dataset] = (df, datasets[curr_dataset][1])
df



C:\Users\danza\AppData\Local\Temp\ipykernel_1308\141281835.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["Sex"],axis=1, inplace=True)
C:\Users\danza\AppData\Local\Temp\ipykernel_1308\141281835.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["School leavers (graduates and dropouts)"],axis=1, inplace=True)
C:\Users\danza\AppData\Local\Temp\ipykernel_1308\376170415.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,Year,Pupils,School beginners
49,2000,9960447,811955
50,2001,9870445,780755
51,2002,9780277,805534
52,2003,9727034,843813
53,2004,9624854,820674
54,2005,9505241,822316
55,2006,9355857,793099
56,2007,9183811,772725
57,2008,9023572,735316
58,2009,8905800,726384


#### Suspects

In [1231]:
curr_dataset = 10
df = datasets[curr_dataset][0]
df = prepend_col_name(df, "Suspects")
datasets[curr_dataset] = (df, datasets[curr_dataset][1])
df.dtypes

Year                          int32
Suspects_Germans_Male         int32
Suspects_Germans_Female       int32
Suspects_Foreigners_Male      int32
Suspects_Foreigners_Female    int32
dtype: object

#### Total criminal offences

In [1232]:
curr_dataset = 11
df = datasets[curr_dataset][0]
datasets[curr_dataset] = (df, datasets[curr_dataset][1])
df

,Year,Registered criminal offences,Cleared-up criminal offences
1,1993,6750613,2957135
2,1994,6537748,2899733
3,1995,6668717,3068379
4,1996,6647598,3255042
5,1997,6586165,3335016
6,1998,6456996,3376524
7,1999,6302316,3329124
8,2000,6264723,3335356
9,2001,6363865,3379618
10,2002,6507394,3425416


#### Unemployment

In [1233]:
curr_dataset = 12
df = datasets[curr_dataset][0]
df = df.groupby("Year", as_index=False).mean()
df.dtypes
datasets[curr_dataset] = (df, datasets[curr_dataset][1])


### Merge all datasets into one

In [1252]:
df1 = mass_join(datasets)
df2 = mass_join_till_year_x(datasets, 2023)

final = pd.read_csv('../../datasets/processed/non-pks/final.csv')
final.drop([final.columns[0]],axis=1, inplace=True)
final.rename(columns={final.columns[0]: "Year"}, inplace=True)

until_2022 = final.merge(df1, how="inner", on="Year")
until_2022
until_2023 =final.merge(df2, how="inner", on="Year")
until_2023
until_2022.to_csv('../../datasets/processed/non-pks/final_until2022.csv', index=False)
until_2023.to_csv('../../datasets/processed/non-pks/final_until2023.csv', index=False)